# Sorting Algorithms Benchmark in Python (Jupyter Workflow)

This notebook guides you through an **empirical comparison** of three sorting algorithms:

- **Insertion Sort** (simple, quadratic-time worst/average)
- **Merge Sort** (divide-and-conquer, O(n log n))
- **Timsort** (Python's built-in `sorted`/`.sort()`), a hybrid of merge sort and insertion sort that is **adaptive** to existing runs

## Objectives

1. Implement the algorithms (in pure Python for insertion/merge; use `sorted` for Timsort).
2. Generate different input patterns: **random**, **already-sorted**, **reversed**, and **nearly-sorted**.
3. Benchmark with `timeit` over increasing array sizes; use the **median** over multiple runs.
4. Visualize results and **validate theoretical complexities** empirically.
5. Show why **Timsort** tends to outperform pure merge sort and insertion sort in practice.

> Run cells from top to bottom. Adjust parameters (sizes, repeats) as needed for your machine.

## 1. Imports & Environment

We rely on `timeit`, `random`, `pandas`, and `matplotlib`. If something is missing, install it before continuing.


In [ ]:
import sys, platform
print(f'Python: {sys.version.split()[0]} on {platform.system()} {platform.release()}')

In [ ]:
import timeit
import random

import pandas as pd
import matplotlib.pyplot as plt

## 2. Sorting Algorithms

- `insertion_sort`: in-place-like behavior but returns a new list (operates on a copy).
- `merge_sort`: classic top-down recursive merge sort, returning a new list.
- `timsort_builtin`: wrapper around Python's `sorted` (i.e., Timsort).


In [ ]:
from typing import List

def insertion_sort(arr: List[int]) -> List[int]:
    a = arr[:]
    for i in range(1, len(a)):
        key = a[i]
        j = i - 1
        while j >= 0 and a[j] > key:
            a[j + 1] = a[j]
            j -= 1
        a[j + 1] = key
    return a

def merge_sort(arr: List[int]) -> List[int]:
    if len(arr) <= 1:
        return arr[:]
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    i = j = 0
    merged = []
    append = merged.append
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            append(left[i]); i += 1
        else:
            append(right[j]); j += 1
    if i < len(left):
        merged.extend(left[i:])
    if j < len(right):
        merged.extend(right[j:])
    return merged

def timsort_builtin(arr: List[int]) -> List[int]:
    return sorted(arr)


## 3. Data Generators

We will benchmark on four patterns:
- **random**
- **already-sorted**
- **reversed**
- **nearly-sorted** (few random swaps applied to a sorted array)


In [ ]:
from typing import Callable, Dict

def gen_random(n: int, seed: int = 42) -> List[int]:
    rnd = random.Random(seed)
    return [rnd.randint(0, 10**9) for _ in range(n)]

def gen_sorted(n: int) -> List[int]:
    return list(range(n))

def gen_reversed(n: int) -> List[int]:
    return list(range(n, 0, -1))

def gen_nearly_sorted(n: int, swaps_ratio: float = 0.05, seed: int = 123) -> List[int]:
    a = list(range(n))
    rnd = random.Random(seed)
    swaps = max(1, int(n * swaps_ratio))
    for _ in range(swaps):
        i = rnd.randrange(n)
        j = rnd.randrange(n)
        a[i], a[j] = a[j], a[i]
    return a

PATTERNS: Dict[str, Callable[[int], List[int]]] = {
    "random": gen_random,
    "sorted": gen_sorted,
    "reversed": gen_reversed,
    "nearly_sorted": gen_nearly_sorted,
}

## 4. Quick Sanity Checks

Verify that our implementations actually sort correctly by comparing with `sorted()`.


In [ ]:
def sanity_check():
    samples = [
        [3,1,4,1,5,9,2,6],
        [],
        [1],
        [2,2,2,1,1,3],
        list(range(10)),
        list(range(10, 0, -1)),
    ]
    algos = [insertion_sort, merge_sort, timsort_builtin]
    for s in samples:
        gold = sorted(s)
        for algo in algos:
            assert algo(s) == gold, f"{algo.__name__} failed on {s}"
    print("All sanity checks passed.")
    
sanity_check()

## 5. Timing Helper (`timeit`)

We measure the **median** time over several repeats to reduce noise.


In [ ]:
from statistics import median
from typing import Callable

def time_algorithm(func: Callable[[List[int]], List[int]], data: List[int], repeat: int = 5) -> float:
    """Return median time (seconds) over `repeat` runs for sorting `data` with `func`."""
    def run():
        out = func(list(data))
        assert out == sorted(data)
    times = timeit.repeat(run, number=1, repeat=repeat)
    return median(times)

## 6. Configure Benchmark Parameters

Adjust sizes and repeats depending on your machine. Insertion sort is **O(n²)**,
so keep its sizes moderate to avoid very long runs.


In [ ]:
ALGORITHMS: Dict[str, Callable[[List[int]], List[int]]] = {
    "insertion": insertion_sort,
    "merge": merge_sort,
    "timsort": timsort_builtin,
}

SIZES = [200, 400, 800, 1600, 3200, 6400]

REPEATS = 5

SIZES, REPEATS

## 7. Run Benchmarks

This will generate a Pandas DataFrame with results across patterns, sizes, and algorithms.


In [ ]:
results = []
for pattern_name, generator in PATTERNS.items():
    for n in SIZES:
        data = generator(n)
        for algo_name, algo in ALGORITHMS.items():
            if algo_name == "insertion" and n > 6400:
                continue
            t = time_algorithm(algo, data, repeat=REPEATS)
            results.append({
                "pattern": pattern_name,
                "n": n,
                "algorithm": algo_name,
                "time_sec": t,
            })

df = pd.DataFrame(results).sort_values(["pattern", "n", "algorithm"]).reset_index(drop=True)
df

## 8. Plot Results

One figure per data pattern. We keep the plotting simple and consistent with the constraints.


In [ ]:
from pathlib import Path

def plot_pattern(df_pattern: pd.DataFrame, pattern_name: str) -> Path:
    plt.figure(figsize=(7, 5))
    for algo_name in sorted(df_pattern["algorithm"].unique()):
        sub = df_pattern[df_pattern["algorithm"] == algo_name]
        plt.plot(sub["n"], sub["time_sec"], marker="o", label=algo_name)
    plt.xlabel("n (array size)")
    plt.ylabel("Time (seconds, median of runs)")
    plt.title(f"Sorting performance on '{pattern_name}' data")
    plt.legend()
    out_path = Path(f"sort_perf_{pattern_name}.png")
    plt.savefig(out_path, bbox_inches="tight", dpi=150)
    plt.show()
    return out_path

image_paths = []
for pattern in PATTERNS.keys():
    image_paths.append(plot_pattern(df[df["pattern"] == pattern], pattern))

image_paths

## 9. Analysis

**Theory recap:**  
- Insertion sort: **O(n²)** average/worst, but **O(n)** on already-sorted data.  
- Merge sort: **O(n log n)** in all cases.  
- Timsort: **O(n log n)** worst/average, but **adaptive**: very fast on partially sorted data by exploiting natural runs and using insertion sort on small runs.

**Empirical observations to look for in the plots:**  
- Insertion sort grows quadratically on random/reversed inputs and becomes much slower than the others as `n` increases.  
- Merge sort shows near-linearithmic scaling and is consistently competitive.  
- Timsort generally outperforms merge sort (often by a notable margin), and can be especially strong on sorted/nearly-sorted inputs.

In practice `sorted()`/`.sort()` is preferable because it is **highly optimized, adaptive, and stable**, combining the strengths of merge and insertion sorts.
